In [81]:
from pathlib import Path
from sys import stdout

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
from ruamel.yaml import YAML
from sklearn.linear_model import LinearRegression

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}
yaml = YAML()

kB = 1000 * Boltzmann / electron_volt

import os
#os.getcwd()
#os.chdir('data')
#os.chdir('/home/ssilayi/Projects/spyns/Fe/exchange_model/')


In [82]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)

fe_magnetic_patterns = yaml.load(Path("data/fe_magnetic_patterns.yml"))

In [83]:
#x={}
N = 4
x['nm'] = fe_magnetic_patterns['4atoms']['nm']
x['f-type'] = fe_magnetic_patterns['4atoms']['f-type']
x['a-type'] = fe_magnetic_patterns['4atoms']['a-type']
x['g-type'] = fe_magnetic_patterns['4atoms']['g-type']
fe_new = fe_structure_4atom.copy(x)
fe_new.make_supercell([N,N,N])

In [84]:
fe_supercell = pd.concat([pd.DataFrame(fe_new.cart_coords, columns=["#x","y","z"]), 
                          pd.DataFrame(fe_new.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(fe_new.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(fe_new.site_properties["a-type"], columns=["a-type"]) ,
                          pd.DataFrame(fe_new.site_properties["g-type"], columns=["g-type"]) ],
                          axis=1, )
fe_supercell.style.format('{:.2f}')
fe_supercell

,#x,y,z,nm,f-type,a-type,g-type
0,0.000,0.000,0.000,0,1,1,1
1,0.000,0.000,2.866,0,1,1,1
2,0.000,0.000,5.733,0,1,1,1
3,0.000,0.000,8.599,0,1,1,1
4,14.332,8.599,11.466,0,1,1,1
5,14.332,8.599,2.867,0,1,1,1
6,14.332,8.599,5.733,0,1,1,1
7,14.332,8.599,8.600,0,1,1,1
8,17.199,5.733,11.466,0,1,1,1
9,17.199,5.733,2.867,0,1,1,1


In [85]:
#n_sites = len(fe_supercell)
filename = 'fe_supercell.txt'
f=open(filename,'w')
#f.write(str(n_sites)+"\n" )
fe_supercell.to_csv(filename,index=None, mode='a', sep=' ')
f.close()